In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
def load_pkl(fname):
    with open(fname, 'rb') as infile:
        return pickle.load(infile)
def save_pkl(df, fname):
    with open(fname, 'wb') as outfile:
        pickle.dump(df, outfile)
def na_percent(df):
    return df.isna().sum()/len(df)
def map_customer_id(df):
    df['customer_id'] = df['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')
def map_article_id(df):
    df['article_id'] = df['article_id'].astype('int32')
    # when making predictions, should convert to string and add leading 0
    # e.g. train['article_id'] = '0' + train.article_id.astype('str')
path='/content/drive/MyDrive/Colab Notebooks/'

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
#df_articles = pd.read_pickle(path+'df_articles.pkl')
#df_customers = load_pkl(path+'df_customers.pkl')
df_transactions_train = load_pkl(path+'df_transactions_train.pkl')

In [5]:
df_sample_submission = load_pkl(path+'df_sample_submission.pkl')

In [ ]:
df_customers.head()

In [7]:
df_articles.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [8]:
df_transactions_train.head()

,customer_id,article_id,price,sales_channel_id,year,month,day
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2,18,9,20
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2,18,9,20
2,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2,18,9,20
3,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2,18,9,20
4,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2,18,9,20


In [6]:
df_transactions_train["day_num"] = df_transactions_train["year"]*365 + df_transactions_train["month"]*30 + df_transactions_train["day"]

In [7]:
df_transactions_train["day_dist"] = (df_transactions_train["day_num"].max() - df_transactions_train["day_num"])

In [8]:
most_frequent_items = df_transactions_train.article_id.value_counts().index[:12]

In [9]:
most_frequent_items

Int64Index([706016001, 706016002, 372860001, 610776002, 759871002, 464297007,
            372860002, 610776001, 399223001, 706016003, 720125001, 156231001],
           dtype='int64')

In [10]:
cold_lst = list(set(df_sample_submission.customer_id.unique()) - set(df_transactions_train.customer_id.unique()))

In [11]:
len(cold_lst)

9699

In [15]:
df_tmp = df_transactions_train.sort_values(["customer_id", "day_num"], ascending=False)

In [16]:
def keep_latest_k(articles, k=12):
    result = []
    # Use most commonly bought items to fill the empty spots
    for item in articles:
        if item in result:
            continue
        result.append(item)
        if len(result) == k:
            break
    for item in most_frequent_items:
        if item in result:
            continue
        result.append(item)
        if len(result) == k:
            break
    return result

In [17]:
df_latest_items = df_tmp.groupby("customer_id").agg({"article_id": keep_latest_k}).reset_index()
df_latest_items["prediction"] = df_latest_items.article_id.apply(lambda x: " ".join(map("{:010d}".format, x)))
df_latest_items.head()

,customer_id,article_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,"[841260003, 887593002, 890498002, 795440001, 8...",0841260003 0887593002 0890498002 0795440001 08...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,"[599580055, 811835004, 723529001, 559630026, 5...",0599580055 0811835004 0723529001 0559630026 05...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,"[858883002, 851400006, 750424014, 870304002, 8...",0858883002 0851400006 0750424014 0870304002 08...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,"[742079001, 732413001, 706016001, 706016002, 3...",0742079001 0732413001 0706016001 0706016002 03...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,"[896152002, 730683050, 927530004, 791587015, 5...",0896152002 0730683050 0927530004 0791587015 05...


In [22]:
total_df = pd.concat([df_latest_items[["customer_id", "prediction"]], df_sample_submission[df_sample_submission['customer_id'].isin(cold_lst)]])
total_df

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0841260003 0887593002 0890498002 0795440001 08...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0599580055 0811835004 0723529001 0559630026 05...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0858883002 0851400006 0750424014 0870304002 08...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0742079001 0732413001 0706016001 0706016002 03...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0896152002 0730683050 0927530004 0791587015 05...
...,...,...
1371440,ffe5801cb2a5b51d4d068322d7f8082e995f427a6f22a6...,0706016001 0706016002 0372860001 0610776002 07...
1371554,ffeb3ca867aba57a312fe9d28d67dd46ef2240fe92a94c...,0706016001 0706016002 0372860001 0610776002 07...
1371739,fff456fa60aac9174456c2f36ede5e0f25429a16c88a34...,0706016001 0706016002 0372860001 0610776002 07...
1371872,fffa8d3cea26d4f5186472b923629b35fa28051f258030...,0706016001 0706016002 0372860001 0610776002 07...


In [26]:
total_df.to_csv(path+'submission1.csv', index=False)

In [6]:
import pandas as pd, numpy as np
train = load_pkl(path + 'df_transactions_train.pkl')
pairs = np.load(path + 'pairs_cudf.npy',allow_pickle=True).item()
train['article_id2'] = train.article_id.map(pairs)

In [7]:
train2 = train[['customer_id','article_id2']].copy()
train2 = train2.loc[train2.article_id2.notnull()]
train2 = train2.drop_duplicates(['customer_id','article_id2'])
train2 = train2.rename({'article_id2':'article_id'},axis=1)

In [8]:
train = train[['customer_id','article_id']]
train = pd.concat([train,train2],axis=0,ignore_index=True)
train.article_id = train.article_id.astype('int32')
train = train.drop_duplicates(['customer_id','article_id'])

In [9]:
train.article_id = ' 0' + train.article_id.astype('str')
preds = pd.DataFrame(train.groupby('customer_id').article_id.sum().reset_index())
preds.columns = ['customer_id','prediction']
preds

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0625548001 0176209023 0627759010 0697138006 0...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0583558001 0639677008 0640244003 0521269001 0...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001 0541518023 0578020002 0723529001 0...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0742079001 0732413001 0730683001 0372860001
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0634249005 0677049001 0698286003 0707704003 0...
...,...,...
1362276,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,0698276003 0699075005 0694182002 0722436003 0...
1362277,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,0671695001 0562245015 0562245018 0562245004 0...
1362278,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,0568597019 0484398001 0701083001 0597043002 0...
1362279,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,0821395003 0806241002 0714790020 0866755002 0...


In [10]:
save_pkl(preds, path+'pred_association.pkl')

In [8]:
preds = load_pkl(path + 'pred_association.pkl')
train = load_pkl(path + 'df_transactions_train.pkl')
train["day_num"] = train["year"]*365 + train["month"]*30 + train["day"]

In [10]:
train = train.loc[train.day_num >= 7440]
top12 = ' 0' + ' 0'.join(train.article_id.value_counts().index.astype('str')[:12])
top12

' 0741356002 0720125001 0599580055 0610776001 0759871001 0841383002 0841383003 0759465001 0610776002 0824337001 0766346008 0684209013'

In [26]:
sub = load_pkl(path + 'df_sample_submission.pkl')
sub = sub[['customer_id']]
sub = pd.merge(sub, preds, how='left')
sub.prediction = sub.prediction + top12
sub.prediction = sub.prediction.str.strip()
sub.prediction = sub.prediction.str[:131]
sub

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0625548001 0176209023 0627759010 0697138006 05...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0583558001 0639677008 0640244003 0521269001 06...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001 0541518023 0578020002 0723529001 03...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0742079001 0732413001 0730683001 0372860001 07...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0634249005 0677049001 0698286003 0707704003 03...
...,...,...
1371975,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,0698276003 0699075005 0694182002 0722436003 04...
1371976,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,0671695001 0562245015 0562245018 0562245004 06...
1371977,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,0568597019 0484398001 0701083001 0597043002 06...
1371978,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,0821395003 0806241002 0714790020 0866755002 08...


In [27]:
sub.to_csv(path+'submission3.csv', index=False)